In [1]:
from selenium import webdriver

from bs4 import BeautifulSoup as bs
import bs4
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
from lxml import etree
import csv

### 스크래퍼 클래스 선언

In [7]:
class WebtoonParser(object):
    def __init__(self):
        self.driver = webdriver.Chrome()
        ### 월,화,수,목,금,토,일,매일,완결
        self.weekend = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun','dailyPlus','finish']
        #self.weekend = ['mon']
        self.url = 'https://comic.naver.com/webtoon?tab='
        self.web = []
        self.image = []
        self.webtoon_url = {}

    def excute(self):
        for category in self.weekend:
            nw_url = self.url + category
            driver = self.driver
            driver.get(nw_url)

            sleep(1)
            # 스크롤 높이 가져옴
            self.scrolling(driver)

            sleep(1)

            html = driver.page_source
            soup = bs(html, 'html.parser')

            ### 이미지 추출
            for title in soup.findAll('li', {'class':'item'}):
                self.recur_webtoon(title)
                img= title.findAll('img')
                # print("이미지 탐색 중")
                self.image.extend(img)
                # print("이미지 탐색 완료")
            

            webtoon_list = soup.find('ul', {'class':'ContentList__content_list--q5KXY'})
            self.total = len(webtoon_list)

    ### 타이틀 , 작가, 평점 추출
    def recur_webtoon(self,tag):
        for child in tag.descendants:
            if type(child) == bs4.element.NavigableString:
                if child == '청유물' or child == '신작' or child == '휴재' or child == '완결' or child == 'UP' or child == 'NEW' or child == '유료작품':
                    continue
                self.web.append(child)
            else:
                id_ = child.find('a')
                if id_ == None:
                    continue
                if id_['class'][0] == 'ContentTitle__title_area--x24vt':
                    id_text = id_.find('span', {'class':'ContentTitle__title--e3qXt'}).text
                    self.webtoon_url[id_text] = id_['href']
                    

    def scrolling(self, driver):
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_division = 10
        while True:
            
            print("스크롤 중")
            # 끝까지 스크롤 다운
            for i in range(1,scroll_division+1):
                driver.execute_script(f"window.scrollTo(0,(document.body.scrollHeight/{scroll_division})*{i})")
                sleep(1)
            sleep(1)
            # 스크롤 다운 후 스크롤 높이 다시 가져옴
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height



### 실행

In [8]:
abc = WebtoonParser()
abc.excute()

스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중
스크롤 중


### title, author, rating dictionary 저장

In [11]:
lt1 = abc.web

webtoon = []
for i in range(0,len(lt1),4):
    dic = {}
    try:    
        dic['title'] = lt1[i]
        dic['author'] = lt1[i+1]
        dic['rating'] = lt1[i+3]
    except:
        print("abc")

    webtoon.append(dic)

### title image 추가

In [13]:
for a, i in enumerate(abc.image):
    for b, j in enumerate(webtoon):
        if i['alt'] == j['title']:
            webtoon[b]['image'] = i['src']
            

### url 추가

In [14]:
for a, i in enumerate(abc.webtoon_url):
    for b, j in enumerate(webtoon):
        if i == j['title']:
            webtoon[b]['url'] = abc.webtoon_url[i]

### 제목 [드라마원작] 제거

In [20]:
for a,i in enumerate(webtoon):
    try:
        webtoon[a]['title'] = webtoon[a]['title'].replace("[드라마원작] ", "")
    except:
        webtoon[a]['url'] = "none"

### csv 파일로 저장

In [21]:
with open('./data/webtoon.csv', 'w', encoding='utf-8', newline='') as f:
    fieldnames = ['title', 'author', 'rating', 'image', 'url']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for i in webtoon:
        writer.writerow(i)
        


### 장르 크롤링

In [33]:

webtoon_list = []

with open('webtoon.csv', 'r', encoding='utf-8', newline='') as f:
    fieldnames = ['title', 'author', 'rating', 'image','url']
    reader = csv.DictReader(f, fieldnames=fieldnames)
    
    for n, row in enumerate(reader):
        if n == 0:
            continue

        tmp = {}
        tmp['title'] = row['title']
        tmp['author'] = row['author']
        tmp['rating'] = row['rating']
        tmp['image'] = row['image']
        url = 'https://comic.naver.com'+ row['url']
        tmp['url'] = url

        diver = webdriver.Chrome()
        diver.get(url)
        sleep(1)

        html = diver.page_source
        soup = bs(html, 'html.parser')
        
        try:
            a = soup.find('div', {'class':'TagGroup__tag_group--uUJza'}).text
        except:
            tmp['genre'] = '청소년이용불가'
            webtoon_list.append(tmp)
            diver.close()
            continue

        tmp['genre'] = a

        webtoon_list.append(tmp)

        diver.close()


### 최종 형태 저장

In [ ]:
with open('./data/webtoon_com.csv', 'w', encoding='utf-8', newline='') as f1:
    fieldnames1 = ['title', 'author', 'rating', 'image', 'url','genre']
    writer1 = csv.DictWriter(f1, fieldnames=fieldnames1)
    writer1.writeheader()
    for i in webtoon_list:
        writer1.writerow(i)